In [1]:
import numpy as np
import pandas as pd
import json
import requests

In [2]:
id_base = {'cater': 1903,
          'hospAd': 503,
          'hospCh': 505,
          'edu': 2263,
          'wifi':2756,
          'firehelp':1949}

In [91]:
result = {}
for dataset in id_base:
    url = 'https://apidata.mos.ru/v1/datasets/ID_DATASET/method?\
projection=null&api_key=5ffbe7a6b51c789900552b0ffd6c3f83'
    url1 = url.replace('method', 'count')
    resp = requests.get(url1.replace('ID_DATASET', str(id_base[dataset])))
    rownum = int(resp.text)
    print('rows count is ', rownum)
    res_list = []
    if rownum > 1e4:
        skip = 0
        count = len(res_list)
        while(count != rownum):
            url2 = url + '&$top=500&$skip='+str(skip)
            resp2 = requests.get(url2.replace('ID_DATASET', str(id_base[dataset])).replace('method','rows'))
            res_list += resp2.json()
            count = len(res_list)
            print('count is ', count)
            skip +=500
    if len(res_list) == 0:
        res_list = requests.get(url.replace('ID_DATASET', str(id_base[dataset])).replace('method', 'rows')).json()
    result[dataset] = res_list

rows count is  16483
count is  500
count is  1000
count is  1500
count is  2000
count is  2500
count is  3000
count is  3500
count is  4000
count is  4500
count is  5000
count is  5500
count is  6000
count is  6500
count is  7000
count is  7500
count is  8000
count is  8500
count is  9000
count is  9500
count is  10000
count is  10500
count is  11000
count is  11500
count is  12000
count is  12500
count is  13000
count is  13500
count is  14000
count is  14500
count is  15000
count is  15500
count is  16000
count is  16483
rows count is  276
rows count is  208
rows count is  602
rows count is  1422
rows count is  715


In [101]:
with open('res.json', 'w') as outfile:
    json.dump(result, outfile)

In [2]:
with open('res.json', 'r') as outfile:
    data = json.load(outfile)

# Предобработка данных
Распарсим данные для удобной работы с адресами. Создадим 2 словаря:<br>
1) Улица - район. Ключ улица, значение район.<br>
2) Район - округ. Ключ район, значение округ.<br>
Это понадобится для быстрого определения значений<br>

In [4]:
data.keys()

dict_keys(['cater', 'hospAd', 'hospCh', 'edu', 'wifi', 'firehelp'])

In [29]:
street_ray = {}
ray_adm = {}
main_base = {}

In [30]:
def addNewInfo(base, adm, ray, street, type_info, info):
    if adm not in base:
        base[adm] = {}
        base[adm][ray] = {}
        base[adm][ray][type_info] = []
        base[adm][ray][type_info].append(info)
    else:
        dic_ray = base[adm]
        if ray not in dic_ray:
            dic_ray[ray] = {}
            dic_ray[ray][type_info] = []
            dic_ray[ray][type_info].append(info)
        else:
            dic_type = dic_ray[ray]
            if type_info not in dic_type:
                dic_type[type_info] = []
                dic_type[type_info].append(info)
            else:
                dic_type[type_info].append(info)

In [31]:
for i in data['cater']:
    adm = i['Cells']['AdmArea']
    ray = i['Cells']['District']
    addres = i['Cells']['Address']
    adl = addres.split(',')
    realad = adl[1]
    if adl[0] != 'город Москва':
        realad = adl[0]
    if realad[0] == ' ':
        realad = realad[1:]
    street_ray[realad] = ray
    ray_adm[ray] = adm
    addNewInfo(main_base, adm, ray, realad, 'cater', i)

In [44]:
for i in data['hospAd']:
    adm = i['Cells']['ObjectAddress'][0]['AdmArea']
    ray = i['Cells']['ObjectAddress'][0]['District']
    fulladress = i['Cells']['ObjectAddress'][0]['Address']
    adl = fulladress.split(',')
    realad = adl[1]
    if adl[0] != 'город Москва':
        realad = adl[0]
    elif 'поселок' in adl[1] or 'поселение' in adl[1] or 'город' in adl[1]:
        realad = adl[2]
    if realad[0] == ' ':
        realad = realad[1:]
    street_ray[realad] = ray
    ray_adm[ray] = adm
    addNewInfo(main_base, adm, ray, realad, 'hospAd', i)

In [60]:
for i in data['hospCh']:
    adm = i['Cells']['ObjectAddress'][0]['AdmArea']
    ray = i['Cells']['ObjectAddress'][0]['District']
    fulladress = i['Cells']['ObjectAddress'][0]['Address']
    adl = fulladress.split(',')
    realad = adl[1]
    if adl[0] != 'город Москва':
        realad = adl[0]
    elif 'поселок' in adl[1] or 'поселение' in adl[1] or 'город' in adl[1]:
        realad = adl[2]
    if realad[0] == ' ':
        realad = realad[1:]
    street_ray[realad] = ray
    ray_adm[ray] = adm
    addNewInfo(main_base, adm, ray, realad, 'hospCh', i)

In [71]:
for i in data['edu']:
    adm = i['Cells']['InstitutionsAddresses'][0]['AdmArea']
    ray = i['Cells']['InstitutionsAddresses'][0]['District']
    fulladress = i['Cells']['InstitutionsAddresses'][0]['Address']
    adl = fulladress.split(',')
    realad = adl[1]
    if adl[0] != 'город Москва':
        realad = adl[0]
    elif 'поселок' in adl[1] or 'поселение' in adl[1] or 'город' in adl[1]:
        realad = adl[2]
    if realad[0] == ' ':
        realad = realad[1:]
    street_ray[realad] = ray
    ray_adm[ray] = adm
    addNewInfo(main_base, adm, ray, realad, 'edu', i)

In [76]:
for i in data['wifi']:
    adm = i['Cells']['AdmArea']
    ray = i['Cells']['District']
    fulladress = i['Cells']['Location']
    adl = fulladress.split(',')
    realad = adl[1]
    if adl[0] != 'город Москва':
        realad = adl[0]
    elif 'поселок' in adl[1] or 'поселение' in adl[1] or 'город' in adl[1]:
        realad = adl[2]
    if realad[0] == ' ':
        realad = realad[1:]
    street_ray[realad] = ray
    ray_adm[ray] = adm
    addNewInfo(main_base, adm, ray, realad, 'wifi', i)

In [87]:
with open('main.json', 'w') as outfile:
    json.dump(main_base, outfile)
with open('street_ray.json', 'w') as outfile:
    json.dump(street_ray, outfile)
with open('ray_adm.json', 'w') as outfile:
    json.dump(ray_adm, outfile)

In [83]:
def getLevelLife(main_base, street_ray, ray_adm):
    inp = 1
    while(inp):
        add = input('Enter the street name: ')
        if add in street_ray:
            inp = 0
    ray = street_ray[add]
    adm = ray_adm[ray]
    count_food = len(main_base[adm][ray]['cater'])
    count_hospCh = len(main_base[adm][ray]['hospCh'])
    count_edu = len(main_base[adm][ray]['edu'])
    count_year = 0
    count_adm = 0
    for i in data['firehelp']:
        cell = i['Cells']
        if cell['Year'] == 2019:
            count_year += cell['Calls']
            if cell['AdmArea'] == adm:
                count_adm += cell['Calls']
    return (count_hospCh + count_edu + count_food)/(count_adm/count_year)
    

In [85]:
street_ray.keys()

dict_keys(['улица Егора Абакумова', 'улица Талалихина', 'Абельмановская улица', 'Абрамцевская улица', 'Угличская улица', 'переулок Сивцев Вражек', 'Авиамоторная улица', 'улица Авиаторов', 'Азовская улица', 'Большая Академическая улица', 'Филипповский переулок', 'Алтайская улица', 'Алтуфьевское шоссе', 'улица Амундсена', 'Анадырский проезд', 'Ангарская улица', 'проспект Андропова', 'улица Арбат', 'улица Артамонова', 'Астраханский переулок', 'Аэродромная улица', 'улица Ивана Бабушкина', 'Байкальская улица', 'Бакинская улица', 'Балаклавский проспект', 'улица Барклая', 'Новая Басманная улица', 'Бауманская улица', 'Татарская улица', 'Башиловская улица', 'Беговая улица', 'улица Демьяна Бедного', 'Белореченская улица', 'Бережковская набережная', 'Бескудниковский бульвар', 'улица Бестужевых', 'Бибиревская улица', 'улица Маршала Бирюзова', 'Бойцовая улица', 'Болотниковская улица', 'Коктебельская улица', 'Ботаническая улица', '1-й Боткинский проезд', 'улица Академика Бочвара', '2-я Брестская ули

In [86]:
getLevelLife(main_base, street_ray, ray_adm)

Enter the street name: Чермянская улица


711.1439797923099

# Витрина данных.
К этому моменту я построил базу данных иерархической структурой. Распишем сверху вниз:<br>
1) Округ.<br>
2) Район.<br>
3) Выбор соответсвущей категории: общепит, образование, поликлиники и тд.<br>
4) Список соответствующих объектов.<br>

# Работа с онлайн базой


В дополнение к формуле представленной в задании: я решил добавит в числитель кол-во музеев, домов культуры и спортивных объектов в районе.
Для того, чтобы убрать сильный разброс по месяцам, я решил сделать дифференциальное преобразование: diff = G[t] - G[t-12]<br>
1) https://data.mos.ru/opendata/7702155262-doma-kultury-i-kluby дома культуры <br>
2) https://data.mos.ru/opendata/7702155262-muzei музеи <br>
3) https://data.mos.ru/opendata/7708308010-sportivnye-obekty-goroda-moskvy спорт. комплс. <br>

In [96]:
dop_base = {'dom_k' : 493,
            'mus':529,
            'sport':629}

In [99]:
new_fire = {}
for i in data['firehelp']:
    cell = i['Cells']
    if cell['Year'] == 2019:
        adm = cell['AdmArea']
        m = cell['Month']
        if adm not in new_fire:
            new_fire[adm] = {}
            new_fire[adm][m] = np.log(cell['Calls'])
        else:
            if m not in new_fire[adm]:
                new_fire[adm][m] = np.log(cell['Calls'])

In [102]:
new_fire

{'Центральный административный округ': {'Январь': 5.934894195619588,
  'Февраль': 6.401917196727186,
  'Март': 6.194405391104672,
  'Апрель': 6.326149473155099,
  'Май': 6.317164686747284,
  'Июнь': 6.263398262591624,
  'Июль': 6.214608098422191,
  'Август': 6.297109319933935,
  'Сентябрь': 6.309918278226516,
  'Октябрь': 6.222576268071369,
  'Ноябрь': 6.19644412779452,
  'Декабрь': 6.2285110035911835},
 'Северо-Восточный административный округ': {'Январь': 5.973809611869261,
  'Февраль': 6.505784060128229,
  'Март': 6.3561076606958915,
  'Апрель': 6.583409222158765,
  'Май': 6.369900982828227,
  'Июнь': 6.559615237493242,
  'Июль': 6.244166900663736,
  'Август': 6.35088571671474,
  'Сентябрь': 6.536691597591305,
  'Октябрь': 6.2324480165505225,
  'Ноябрь': 6.2878585601617845,
  'Декабрь': 6.476972362889683},
 'Северо-Западный административный округ': {'Январь': 5.796057750765372,
  'Февраль': 6.054439346269371,
  'Апрель': 6.371611847231857,
  'Май': 6.206575926724928,
  'Июль': 6.008

In [130]:
def online(new_fire, dop_base):
    url_new = 'https://apidata.mos.ru/v1/datasets/ID_DATASET/method?\
projection=null&api_key=5ffbe7a6b51c789900552b0ffd6c3f83'
    inp = 1
    while(inp):
        add = input('Enter the street name: ')
        url_search = url_new.replace('ID_DATASET', str(1903)).replace('method', 'rows') + '&$top=500'+\
        '&$filter=Cells/Address eq'+add
        resp = requests.get(url_search).json()
        if len(resp) == 0:
            continue
        adm = resp[0]['Cells']['AdmArea']
        ray = resp[0]['Cells']['District']
        inp = 0
    count_food = len(main_base[adm][ray]['cater'])
    count_hospCh = len(main_base[adm][ray]['hospCh'])
    count_edu = len(main_base[adm][ray]['edu'])
    count_year = 0
    count_adm = 0
    for i in new_fire:
        for k in new_fire[i]:
            if i == adm:
                count_adm += new_fire[i][k]
            count_year += new_fire[i][k]
    res_count = 0
    for i in dop_base:
        url_search = url_new.replace('ID_DATASET', str(dop_base[i])).replace('method', 'rows')
        resp = requests.get(url_search).json()
        count = 0
        for p in resp:
            cell = p['Cells']
            if cell['ObjectAddress'][0]['District'] == ray:
                count += 1
        res_count += count
    return (count_hospCh + count_edu + count_food + res_count)/(count_adm/count_year)

In [131]:
online(new_fire, dop_base)

Enter the street name: таллинская


1076.4972058210678